# OpenCVを利用した葉面積の推定

Pythonで画像を扱う際に、OpenCVというライブラリがよく利用されます。
今回はこのOpenCVを用いて、作物成長段階の指標となる葉面積を推定します。

まずはOpenCVモジュールを読み込みます。
以下のチャンクを実行すると、今動いているPythonの環境にOpenCVの機能が追加されます。

作業しているパソコンにOpenCVがインストールされていない場合は、エラーが出ますので、以下のコマンドでインストールして、再度実行してください。

うまくインストールされれば、以下のコマンドでバージョンが表示されます。

## OpenCVでの画像の表示

まずは画像ファイルをJupyter Notebookに読み込んで表示してみましょう。

画像は、`.ipynb`ファイルと同じフォルダに`data`という名前のフォルダを作成し、その中に保存してください。以下では、[www.pxfuel.com](https://www.pxfuel.com/en/free-photo-qhhuu)から取得した画像を利用しています。

まずはmatplotlibをインポートします。(必要であればmatplotlibを`!pip install matplotlib` からインストールしてください。)

ここでは、`matplotlib`ライブラリ内の`pyploy`というサブモジュールを`plt`という別名でインポートしました。このように別名を使ってインポートすることにより、たとえば`matplotlib.pyplot.imshow()`と書く代わりに、`plt.imshow()`と書くことができるようになります。

import文はプログラムの一番最初にまとめて記述することが慣例ですが、ここでは必要になる箇所で適宜宣言することにします。

では、`matplotlib`の`matshow`関数を使って画像を表示しましょう。

色が変ですね。

これは、OpenCVとmatplotlibでのデフォルトのRGBチャンネルの順序が異なるため、赤と青が入れ替わってしまっているからです。  
OpenCVはBGRの順、matplotlibはRGBの順でカラーを取り扱います。
正しく読み込むには以下のように読み込みます。

`.jpg`ファイルとして保存された写真をPythonで表示することができました。

## 画像データとはなにか
写真や衛星画像のようなデータはラスタデータと呼ばれ、一定の間隔で格子状に区切られた各グリッドに値をもったデータ形式です。それぞれのグリッドのことをピクセルと呼ぶので、ピクセルデータと呼ぶこともあります。上で画像をプロットした際に、X軸とY軸に数値が振られていますが、この数値が画像のピクセルを表しています。通常左上を起点とした座標で表されます。  
ラスタデータと対になるデータはベクタデータです。分野によっては、ベクタデータは、ベクトル、ポリゴンなどとも呼ばれます。

詳しくはESRIのサイトに解説があるので、参考にしてください。

- [ベクターデータとは: www.esrij.com](https://www.esrij.com/gis-guide/gis-datamodel/vector-data/)
- [ラスターデータとは: wwww.esrij.com](https://www.esrij.com/gis-guide/gis-datamodel/raster-data/)

今回読み込んだ画像はラスタデータです。ここではPythonの中でどのように保存されているかを見ていきます。

まずは`cv2.imread()`で読み込んだ`img_bgr`を`print()`関数を用いて表示させてみます。

`print()`関数によって、画像の中のデータが表示されました。画像データは、整数の値が入ったリストが入れ子になった構造をしていることがわかります。

`type()`関数で、データ型を確認します。

`img_bgr`の実態は、`numpy.ndarray`クラスのデータであることが分かりました。
`numpy.ndarray`クラスは、`shape`プロパティを持つので、これを確認してみます。

実は、OpenCVで読みこんんだデータは、各ピクセルにおける輝度が`[青、緑、赤]`の順で保存され、その各ピクセルの値は、画像の左上から右上、さらに上から下の順で3重の入れ子構造で保存されています。そのため、`shape`を確認することで、読み込んだ画像は高さ853ピクセル、幅1280ピクセルのサイズで、3チャンネルの画像であったことがわかりました。

今回の画像の例では、`print()`関数で一行目に表示された`[ 71 205 152]`の値は、左上のピクセルの青色の値が`71`、緑色が`205`、赤色が`152`の値を持っています。値が大きいほど、ディスプレイ上でその色が明るく光るので、このピクセルは緑>赤>青の順の強さで光って、黄緑色であることが推測できます。

`cv2.cvtColor()`関数で変換された`img_rgb`のデータと比べてみましょう。

`cv2.cvtColor()`によって、赤色の値と青色の値が入れ替えられたことが確認できます。

また、`numpy.array`は以下のようにすることで、データの一部を取り出すことができます。

```Python
array[1軸目の範囲, 2軸目の範囲, 3軸目の範囲, ...]
```

今回の画像データは縦、横、色チャンネルの3次元データです。左上の縦100x横200ピクセルに対応する`img_rgb`は次のように取り出すことができます。

この部分だけをプロットしてみましょう。

画像の左上あたりが切り取られていることが確認できました。

## チャンネルの分解
OpenCVにある`split()`関数を用いると、上記の画像形式の`numpy.ndarray`をチャンネルごとに分割することができます。

`cv2.spolit()`関数は結果を長さ3のタプルで返します。返り値がタプルの場合、代入記号`=`の左側にタプルの要素と同じ数の変数を置くことによって、タプルの各要素をそれぞれの変数に代入することができます。これをアンパックといいます。参考:[Pythonの関数で複数の戻り値を返す方法](https://note.nkmk.me/python-function-return-multiple-values/)

`cv2.spolit(img_rgb)`は、一つ目の返り値に赤チャンネル、二つ目に緑チャンネル、三つ目に青チャンネルを返すので、それぞれのチャンネルの値を`r`、`g`、`b`に代入しました。

各チャンネルの`type`、`shape`を確認しましょう。

元の`img_rgb`は3次元の`numpy.ndarray`でしたが、各チャンネルは画像の縦と横の2次元の`numpy.ndarray`形式になったことが確認できました。

`matplotlib.pyplot`を用いて、チャンネルごとに画像としてプロットしてみましょう。

元の画像と各チャンネルの画像を並べてプロットするには以下のように行います。

## HSVへの変換
色を表現する方法は、赤・緑・青の各バンドの強さを示すほかに、色相(**H**ue)・彩度(**S**aturation)・明度(**V**alue)の3つの値で示す方法があります。これをRGB形式に対して、HSV形式といいます。

画像解析を行う際に、RGBからHSVに変換することがよく行われます。RGBはディスプレイ上で色を表現するには都合がよいですが、各ピクセルが「赤っぽい」のかや、「鮮やかな色」なのかといったことがわかりにくい表現形式になっています。今回のように、画像から「緑色」のピクセルを探す場合、RGBの形式で単純にGの値が大きいところを選択すると、白っぽいところ、つまりRもGもBも大きな値のピクセルも選択されてしまいます。逆に緑っぽいピクセルでも陰になっているとGの値が小さくなるので選択することができません。

HSVに変換することで、色相による選択を容易にすることができるようになります。

RGB形式のデータをHSV形式のデータに変換するには、`cv2.cvtColor()`関数を利用します。

`img_hsv`も同様に`h`と`s`と`v`に分割することができます。

作成したHSVデータをプロットしてみましょう。

## 閾値による分類
画像をHSVへ変換することができたので、色相(Hue)の値が特定の範囲にあるピクセルを抽出することで、葉っぱが写っている面積を算出します。

今回はhの値が40から60の値を抽出してみましょう。

元の画像で緑色のピクセルを抽出することができています。作成した`img_bin`が`1`の値の個所が抽出されたピクセルで、上の画像では**白く**表示されています。

抽出されたピクセルを、画像のすべてのピクセル数で割ることで、画像に占めるダイズの葉の割合を計算することができます。

実測した葉面積とこの画像に対するダイズの葉の割合との校正曲線を作成することで、定点観測により撮影した画像から葉面積を推定することができます。

## おまけ
HSVへの画像変換を行わない場合、画像の抽出がどのようになるか確認してみましょう。

`g_min`の値を様々に変えてみても、HSV画像で抽出した方が少ないノイズで抽出できています。

また以下のグラフで、HSVで抽出されたピクセルはRGBの値でどのように表現されているものかを確認することができます。

Gの値のみで抽出することが難しいことがわかりますね。

In [ ]:
import numpy as np
import random
def plot_3d(img, sampling=1000, mask=None, **kwargs):
    xlab, ylab, zlab = 'R', 'G', 'B'
    y,x,z = img.shape
    img_flat = np.reshape(img, (x*y,z))
    if mask is not None:
        mask_flat = np.reshape(mask, (x*y)).astype('bool')
        img_flat = img_flat[mask_flat, :]
        
    if sampling != 0:
        # random sample 
        img_flat = img_flat[np.random.choice(len(img_flat), sampling, replace=False)]
    colors = img_flat / 255
    fig = plt.figure(**kwargs)
    ax = fig.add_subplot(111, projection='3d')
    ax.scatter(xs=img_flat[:,0],
               ys=img_flat[:,1],
               zs=img_flat[:,2],
               s=10,
               c=colors,
               lw=0)
    ax.set_xlabel(xlab)
    ax.set_ylabel(ylab)
    ax.set_zlabel(zlab)
    plt.show()

# matplotlibのインタラクティブプロットを有効化
# うまく行かない場合はコメントアウトしてください。
%matplotlib widget

plot_3d(img = img_rgb,
        mask=img_bin,
        sampling=1000,
        figsize=(10,10))

In [ ]:
# インタラクティブを無効化
%matplotlib inline